In [ ]:
! pip install kaggle

In [ ]:
from google.colab import files
files.upload()

In [6]:
! cp kaggle.json ~/.kaggle/

In [7]:
! chmod 600 ~/.kaggle/kaggle.json

In [8]:
! kaggle datasets download andrewmvd/car-plate-detection

 99% 201M/203M [00:03<00:00, 64.2MB/s]
100% 203M/203M [00:03<00:00, 63.1MB/s]


In [ ]:
!unzip /content/car-plate-detection.zip

In [11]:
import os
from os import listdir
from os.path import isfile, join

image_path = '/content/images'
images = [f.split('Cars')[1].split('.png')[0] for f in listdir(image_path) if isfile(join(image_path, f))]
images = list(map(int,images))
images.sort()
images = ['Cars'+str(i)+'.xml' for i in images]

In [12]:
n_train = round(len(images)*0.7)
train = images[:n_train]

valid_1 = images[n_train:]

n_test = round(len(valid_1)*0.7)
valid = valid_1[:n_test]

test = valid_1[n_test:]

print(train)
print(valid)
print(test)

['Cars0.xml', 'Cars1.xml', 'Cars2.xml', 'Cars3.xml', 'Cars4.xml', 'Cars5.xml', 'Cars6.xml', 'Cars7.xml', 'Cars8.xml', 'Cars9.xml', 'Cars10.xml', 'Cars11.xml', 'Cars12.xml', 'Cars13.xml', 'Cars14.xml', 'Cars15.xml', 'Cars16.xml', 'Cars17.xml', 'Cars18.xml', 'Cars19.xml', 'Cars20.xml', 'Cars21.xml', 'Cars22.xml', 'Cars23.xml', 'Cars24.xml', 'Cars25.xml', 'Cars26.xml', 'Cars27.xml', 'Cars28.xml', 'Cars29.xml', 'Cars30.xml', 'Cars31.xml', 'Cars32.xml', 'Cars33.xml', 'Cars34.xml', 'Cars35.xml', 'Cars36.xml', 'Cars37.xml', 'Cars38.xml', 'Cars39.xml', 'Cars40.xml', 'Cars41.xml', 'Cars42.xml', 'Cars43.xml', 'Cars44.xml', 'Cars45.xml', 'Cars46.xml', 'Cars47.xml', 'Cars48.xml', 'Cars49.xml', 'Cars50.xml', 'Cars51.xml', 'Cars52.xml', 'Cars53.xml', 'Cars54.xml', 'Cars55.xml', 'Cars56.xml', 'Cars57.xml', 'Cars58.xml', 'Cars59.xml', 'Cars60.xml', 'Cars61.xml', 'Cars62.xml', 'Cars63.xml', 'Cars64.xml', 'Cars65.xml', 'Cars66.xml', 'Cars67.xml', 'Cars68.xml', 'Cars69.xml', 'Cars70.xml', 'Cars71.xml', '

In [ ]:
"""

├───images
│   ├───train
│   └───valid
│   └───test
└───label
    ├───train
    └───valid
    └───test
  
"""

In [13]:
from pathlib import Path

data = [train, valid, test]
xml_files_path = '/content/annotations'
dirs = ['/content/images/train', '/content/images/valid', '/content/images/test']

for i, new_path  in enumerate(dirs):
    
    if not os.path.exists(new_path):
        os.makedirs(new_path)
    
    for xml_file in data[i]:
        img_file = xml_file.split('.xml')[0] + '.png'
        Path(image_path + '/' + img_file).rename(new_path + '/'+ img_file)
        Path(xml_files_path + '/' + xml_file).rename(new_path + '/'+ xml_file)

**voc to yolo format**

In [14]:
import glob
import os
import pickle
import xml.etree.ElementTree as ET
from os import listdir, getcwd
from os.path import join

dirs = ['/content/images/test',
        '/content/images/train',
        '/content/images/valid']

classes = ['licence']

def getImagesInDir(dir_path):
    image_list = []
    for filename in glob.glob(dir_path + '/*.png'):
        image_list.append(filename)

    return image_list

def convert(size, box):
    dw = 1./(size[0])
    dh = 1./(size[1])
    x = (box[0] + box[1])/2.0 - 1
    y = (box[2] + box[3])/2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

def convert_annotation(dir_path, output_path, image_path):
    basename = os.path.basename(image_path)
    basename_no_ext = os.path.splitext(basename)[0]

    in_file = open(dir_path + '/' + basename_no_ext + '.xml')
    out_file = open(output_path + basename_no_ext + '.txt', 'w')
    tree = ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)
    
    

    for obj in root.iter('object'):
        difficult = obj.find('difficult').text
        cls = obj.find('name').text
        if cls not in classes or int(difficult)==1:
            continue
        cls_id = classes.index(cls)
        xmlbox = obj.find('bndbox')
        b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
        bb = convert((w,h), b)
        out_file.write(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')

In [15]:
import glob
import os
import pickle
import xml.etree.ElementTree as ET
from os import listdir, getcwd
from os.path import join

label_dirs = ['/content/label/test/',
        '/content/label/train/',
        '/content/label/valid/']


for dir_path, label_dir in zip(dirs,label_dirs):
    full_dir_path = dir_path
    output_path = label_dir
   
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    image_paths = getImagesInDir(full_dir_path)
    list_file = open(full_dir_path + '.txt', 'w')

    for image_path in image_paths:
        list_file.write(image_path + '\n')
        convert_annotation(full_dir_path, output_path, image_path)
    list_file.close()

    print("Finished processing: " + dir_path)


Finished processing: /content/images/test
Finished processing: /content/images/train
Finished processing: /content/images/valid


In [16]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
from yolov5 import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v6.0-174-g7b31a53 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 42.4/78.2 GB disk)


In [19]:
!python train.py --img 640 --batch 8 --epochs 30 --data coco128.yaml --weights yolov5s.pt --cache

train: weights=yolov5s.pt, cfg=, data=coco128.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=30, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.0-174-g7b31a53 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1, scale=0.5, 